In [34]:

import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive/')
tr_mcc_codes = pd.read_csv(r"/content/drive/MyDrive/ML/tr_mcc_codes.csv", sep = ';')
tr_types = pd.read_csv(r"/content/drive/MyDrive/ML/tr_types.csv", sep = ';')
transactions = pd.read_csv(r"/content/drive/MyDrive/ML/transactions.csv", nrows = 1000000)
gender_train = pd.read_csv(r"/content/drive/MyDrive/ML/gender_train.csv")
print(tr_types)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
     tr_type                                     tr_description
0       3200  Плата за предоставление услуг посредством моби...
1       3210  Плата за предоставление отчета по счету карты ...
2       3800  Плата за обслуживание банковской карты (за пер...
3       4000            Плата за получение наличных в Сбербанке
4       4001  Плата за получение наличных в Сбербанке (в дру...
..       ...                                                ...
150     2990                             Списание по требованию
151     2992  Списание средств для погашения задолженности п...
152     3001            Комиссия за обслуживание ссудного счета
153     3030  Плата за обслуживание банковской карты (за пос...
154     3120                Плата за досрочный перевыпуск карты

[155 rows x 2 columns]


## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'

Без использования merge!

In [2]:
sam = transactions['tr_type'].sample(n=1000)

In [3]:
t = tr_types['tr_description'].str.contains('POS|АТМ', regex=True)
print(sam[sam.isin(tr_types[t].tr_type)].count() / 1000)

0.608



## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [4]:
task2 = pd.DataFrame(transactions['tr_type'].value_counts().reset_index())
task2 = task2.rename(columns={'tr_type':'value', 'index':'tr_type'})
task2 = task2.head(10).merge(tr_types)
task2

,tr_type,value,tr_description
0,1010,231117,Покупка. POS ТУ СБ РФ
1,2010,151166,Выдача наличных в АТМ Сбербанк России
2,7070,149006,Перевод на карту (с карты) через Мобильный бан...
3,1110,137658,Покупка. POS ТУ Россия
4,1030,118975,Оплата услуги. Банкоматы СБ РФ
5,2370,49830,Списание с карты на карту по операции <перевод...
6,7010,28972,Взнос наличных через АТМ (в своем тер.банке)
7,7030,26078,Перевод на карту (с карты) через АТМ (в предел...
8,7071,15781,Перевод на карту (с карты) через Мобильный бан...
9,1100,15476,Покупка. ТУ Россия


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [5]:
data = transactions[['customer_id','amount']]
data = data.groupby(['customer_id'])['amount'].agg(
    [('positive', lambda x : x[x > 0].sum()),
     ('negative', lambda x : x[x < 0].sum())])
data = data.sort_values(by='positive', ascending=False)
print(data)
client_pos = data.iloc[0] 
data = data.sort_values(by='negative')
print(data)
client_neg = data.iloc[0]
print(abs(abs(client_neg['negative']) - client_pos['positive']))
client_pos = client_pos.name
client_neg = client_neg.name
print(client_neg, client_pos)

                 positive      negative
customer_id                            
70780820     1.248115e+09 -1.249952e+09
19950741     5.124225e+08 -1.075056e+09
93472541     5.102598e+08 -5.091702e+08
63233186     4.456971e+08 -4.511752e+08
47853460     4.348520e+08 -4.574018e+08
...                   ...           ...
41173950     0.000000e+00 -1.743473e+06
21567965     0.000000e+00 -6.996679e+06
36489564     0.000000e+00 -6.936923e+06
4303845      0.000000e+00 -1.115138e+07
88449338     0.000000e+00 -1.692320e+07

[1892 rows x 2 columns]
                 positive      negative
customer_id                            
70780820     1.248115e+09 -1.249952e+09
19950741     5.124225e+08 -1.075056e+09
54871257     1.721157e+08 -9.731417e+08
6898820      3.066668e+07 -7.649160e+08
93472541     5.102598e+08 -5.091702e+08
...                   ...           ...
28658492     2.934289e+05 -4.576974e+05
48087576     0.000000e+00 -3.482465e+05
21546709     6.704058e+05 -3.172021e+05
75010111     1.

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [7]:
#prt1
mean1 = pd.DataFrame(
    transactions.groupby(['tr_type'])['amount'].mean().loc[task2['tr_type']])
mean1 = mean1.rename(columns={'amount':'mean'})
median1 = pd.DataFrame(
    transactions.groupby(['tr_type'])['amount'].median().loc[task2['tr_type']])
median1 = median1.rename(columns={'amount':'median'})


#prt2
mean2 = transactions[transactions.customer_id == client_pos].groupby(
    ['tr_type'])['amount'].mean()
median2 = transactions[transactions.customer_id == client_pos].groupby(
    ['tr_type'])['amount'].median()
print(mean1, '\n', median1)
print(mean2, '\n', median2)

                  mean
tr_type               
1010     -19784.748640
2010    -136077.629325
7070      65569.831700
1110     -32119.330371
1030      -5320.980222
2370    -205418.249032
7010     276391.789596
7030      86104.332909
7071      66806.826623
1100     -44061.827262 
             median
tr_type           
1010      -7411.52
2010     -44918.32
7070      11319.42
1110     -11207.57
1030      -2245.92
2370     -44918.32
7010     112295.79
7030      13951.52
7071       3593.47
1100     -10188.26
tr_type
2010   -2.941056e+06
2330   -2.382398e+06
2370   -2.218505e+06
6110    1.756293e+04
7020    1.465129e+04
7021    2.850067e+04
7030    1.406196e+04
7031    1.921036e+04
7034    1.430648e+04
7040    2.153833e+04
7070    1.384821e+04
7071    1.860057e+04
7074    2.281159e+04
Name: amount, dtype: float64 
 tr_type
2010   -3368873.660
2330   -2245915.770
2370   -2245915.770
6110      10028.010
7020      10319.985
7021      28500.670
7030       8848.910
7031      12992.620
7034      1430

## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [8]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [9]:
data_exp = transactions[transactions.amount < 0].groupby('gender')
mean_exp = data_exp['amount'].mean()
print(abs(mean_exp[0] - mean_exp[1]))
data_inc = transactions[transactions.amount > 0].groupby('gender')
mean_inc = data_inc['amount'].mean()
print(abs(mean_inc[0] - mean_inc[1]))

32718.05492021877
63366.57104801515


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 5 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [33]:
transactions['hour'] = transactions['tr_datetime'].str.extract(' (..):').astype(int)
time = transactions[transactions.hour < 6]
print(time[transactions.amount < 0].groupby('gender').size())
#дополнительное задание
dop_male = pd.DataFrame(
    time[time['gender'] == 0].groupby(
        ['mcc_code']).size().sort_values(ascending=False).head(10))
dop_male = dop_male.reset_index().merge(tr_mcc_codes).rename(columns={ 0: 'Value'})
print(dop_male)
dop_female = pd.DataFrame(
    time[time['gender'] == 1].groupby(
        ['mcc_code']).size().sort_values(ascending=False).head(10))
dop_female = dop_female.reset_index().merge(tr_mcc_codes).rename(columns={ 0: 'Value'})
print(dop_female)

gender
0.0    38827
1.0    43017
dtype: int64
   mcc_code  Value                                    mcc_description
0      5411   9633                  Бакалейные магазины, супермаркеты
1      4814   4638  Звонки с использованием телефонов, считывающих...
2      6011   2956  Финансовые институты — снятие наличности автом...
3      6010   1631   Финансовые институты — снятие наличности вручную
4      4829   1597                                  Денежные переводы
5      5812   1517             Места общественного питания, рестораны
6      5541   1439                            Станции техобслуживания
7      5499   1396  Различные продовольственные магазины — рынки, ...
8      5814   1326                              Рестораны, закусочные
9      5912   1265                                             Аптеки
   mcc_code  Value                                    mcc_description
0      6010  25518   Финансовые институты — снятие наличности вручную
1      5411  10928                  Бакалейн

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
